In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
from tqdm import tqdm

import os
import torch
from scipy.linalg import LinAlgError
import numpy as np
from torch_geometric.data import Batch, HeteroData
from scipy.sparse import coo_array

from sklearn.datasets import make_sparse_spd_matrix
from utils.evaluation import normalize_cons, data_inactive_constraints, data_contraint_heuristic, gurobi_solve_qp

In [3]:
rng = np.random.RandomState(1)

In [4]:
root = 'datasets/qp_250_250_0.02_0.02'
os.mkdir(root)
os.mkdir(os.path.join(root, 'processed'))

FileExistsError: [Errno 17] File exists: 'datasets/qp_250_250_0.02_0.02'

### Generic

In [14]:
density = 0.05
Pdensity = 0.05
nrows = 100
ncols = 100

def surrogate_gen():
    assert max(nrows, ncols) * density > 1

    m, n = min(nrows, ncols), max(nrows, ncols)

    # make sure rows and cols are selected at least once
    rows = np.hstack([np.arange(m), np.random.randint(0, m, (n - m,))])
    cols = np.arange(n)

    # generate the rest
    nnz = int(nrows * ncols * density)
    num_rest = nnz - n

    rows_rest = np.random.randint(0, m, (num_rest,))
    cols_rest = np.random.randint(0, n, (num_rest,))

    values = np.random.randn(nnz)

    A = coo_array((values, (np.hstack([rows, rows_rest]), np.hstack([cols, cols_rest]))), shape=(m, n)).toarray()

    x_feas = np.abs(np.random.randn(ncols))  # Ensure x_feas is non-negative
    b = A @ x_feas + np.abs(np.random.randn(nrows))  # Ensure feasibility

    c = np.random.rand(ncols)
    P = make_sparse_spd_matrix(n_dim=A.shape[1], alpha=1 - Pdensity / 2.,
                               smallest_coef=0.1, largest_coef=0.9, random_state=rng).astype(np.float64)
    return P, A, b, c

bounds = None

# create ineq

In [15]:
ips = []
graphs = []
pkg_idx = 0
success_cnt = 0

max_iter = 15000
num = 10000

pbar = tqdm(range(max_iter))
for i in pbar:
    P, A, b, c = surrogate_gen()
    P = P / np.abs(P).max()
    c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
    A, b = normalize_cons(A, b)
    
    try:
        assert np.linalg.matrix_rank(A) == min(*A.shape)
        assert np.all(np.any(A, axis=1)) and np.all(np.any(A, axis=0))
        solution, duals, _ = gurobi_solve_qp(P, c, A, b)
        assert solution is not None
        
    except (AssertionError, LinAlgError):
        continue
    else:
        if solution is not None:
            inactive_idx = data_inactive_constraints(A, b, solution)
            heur_idx = data_contraint_heuristic(P, A, b, c)

            obj = 0.5 * solution @ P @ solution + c.dot(solution)

            P = torch.from_numpy(P).to(torch.float)
            P_where = torch.where(P)
            
            A = torch.from_numpy(A).to(torch.float)
            b = torch.from_numpy(b).to(torch.float)
            c = torch.from_numpy(c).to(torch.float)
            x = torch.from_numpy(solution).to(torch.float)

            A_where = torch.where(A)
            data = HeteroData(
                cons={
                    'num_nodes': b.shape[0],
                    'x': torch.empty(b.shape[0]),
                     },
                vals={
                    'num_nodes': c.shape[0],
                    'x': torch.empty(c.shape[0]),
                },
                cons__to__vals={'edge_index': torch.vstack(A_where),
                                'edge_attr': A[A_where][:, None]},
                vals__to__vals={'edge_index': torch.vstack(P_where),
                                'edge_attr': P[P_where][:, None]},
                x_solution=x,
                duals=torch.from_numpy(duals).float(),
                obj_solution=torch.tensor(obj).float(),
                q=c,
                b=b,
                inactive_idx=torch.from_numpy(inactive_idx).long(),
                heur_idx=torch.from_numpy(heur_idx).long(),
            )
            success_cnt += 1
            graphs.append(data)

    if len(graphs) >= 1000 or success_cnt == num:
        torch.save(Batch.from_data_list(graphs), f'{root}/processed/batch{pkg_idx}.pt')
        pkg_idx += 1
        graphs = []

    if success_cnt >= num:
        break

    pbar.set_postfix({'suc': success_cnt, 'obj': obj})

  1%|▊                                                                                                                 | 99/15000 [00:00<02:17, 108.45it/s, suc=99, obj=12]


In [ ]:
from data.dataset import LPDataset

In [ ]:
ds = LPDataset(root, 'test')

In [ ]:
data = ds[10]

In [ ]:
from transforms.lp_preserve import AddDumbVariables, OracleDropInactiveConstraint, AddRedundantConstraint, ScaleConstraint, ScaleCoordinate, OracleDropIdleVariable, OracleBiasProblem
from transforms.lp_preserve import ComboPreservedTransforms
from utils.evaluation import recover_qp_from_data

In [ ]:
Q,A,c,b,*_ = recover_qp_from_data(data, np.float64)
solution, duals, _ = gurobi_solve_qp(Q, c, A, b)
0.5 * solution @ Q @ solution + c.dot(solution)

In [ ]:
tf = ComboPreservedTransforms({'OracleDropInactiveConstraint': 0.9,
                               'OracleDropIdleVariable': 0.9,
                               'ScaleConstraint': 1.,
                               'ScaleCoordinate': 1.,
                               'AddRedundantConstraint': 0.5,
                               'AddDumbVariables': 0.5})

In [ ]:
tf = OracleBiasProblem(1.)

In [ ]:
for _ in range(100):
    d1 = tf(data)
    Q, A, c, b, *_ = recover_qp_from_data(d1, np.float64)
    solution, duals, _ = gurobi_solve_qp(Q, c, A, b)
    obj = 0.5 * solution @ Q @ solution + c.dot(solution)
    print(obj)

In [ ]:
for _ in range(100):
    d1 = tf(data)
    Q, A, c, b, *_ = recover_qp_from_data(d1, np.float64)
    solution, duals, _ = gurobi_solve_qp(Q, c, A, b)
    obj = 0.5 * solution @ Q @ solution + c.dot(solution)
    transformed_obj = d1.obj_solution
    print(obj, transformed_obj)